In [ ]:
import cv2
import numpy as np
import socket
import json



def draw_overlay(current_room, current_action,previous_action_overlay):

    overlay_width, overlay_height = int(600), int(400)  # Scale by 2.5

    overlay = np.ones((overlay_height, overlay_width, 3), dtype=np.uint8) * 255  # White background

    overlay = np.ones((overlay_height, overlay_width, 3), dtype=np.uint8) * 255  # White background

    # Draw house (outer rectangle)
    cv2.rectangle(overlay, (0, 0), (overlay_width, overlay_height), (0, 0, 0), 4)  # Thicker black border

    # Define room coordinates
    rooms = {
        "Kitchen": (100, 200, 100, 160),
        "Bathroom": (100, 100, 100, 100),
        "Bedroom": (200, 100, 200, 100),
        "Living Room": (400, 100, 100, 260),
    }

    actions = {
        "Fridge": (101, 270, 25, 25),
        "Laying on Bed": (270, 101, 60, 60),
        "Sink": (169, 130, 30, 40),
        "Sitting on Couch": (469, 200, 30, 80),
    }


    # Draw rooms
    for room_name, (x, y, w, h) in rooms.items():
        color = (154, 205, 50)  # Medium sea green
        if room_name == current_room:
            color = (0, 165, 255)  # Orange highlight
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)  # Filled
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (255, 255, 255), 1)  # Black border
        cv2.putText(overlay, room_name, (x + 5, y + h - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (120, 120, 120), 1)

    # Draw actions
    for action_name, (x, y, w, h) in actions.items():
        color = (128, 128, 128)  # Grey
        if action_name == current_action:
            color = (0, 255, 255)  # Yellow highlight
        if action_name == previous_action_overlay:
            color = (0, 120, 120)
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)  # Filled
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 0), 1, lineType=cv2.LINE_AA)

    return overlay

server_ip = "127.0.0.1"  # Loopback address for same-machine connections
server_port = 5000

#TIMEOUT = 20
buffer_size = 1024

sockconnect = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sockconnect.connect((server_ip, server_port)) 
#sockconnect.settimeout(TIMEOUT)

print(f"Server started on IP: {server_ip} and Port: {server_port}")
#recieved_data = bytearray()


while True:
    response = sockconnect.recv(buffer_size)
    data = json.loads(response.decode('utf-8'))
    print(data)

    current_room = data["room"]
    current_action = data["action"]
    previous_action_overlay = data["previous_action"]

    frame = draw_overlay(current_room, current_action, previous_action_overlay)
    cv2.imshow("Live Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()



ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it